# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
# Load the dotenv extension
%load_ext dotenv

# Load environment variables from .env
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [7]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:
import os
from glob import glob

# Load the environment variable PRICE_DATA
ft_dir = os.getenv('PRICE_DATA')

# If recursive is true, the pattern “**” will match any files and zero or more directories, subdirectories and symbolic links to directories.
parquet_files = glob(ft_dir + '**/*.parquet', recursive=True)
parquet_files

['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Reading multiple parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files)

# Grouping by Ticker column and group_keys = False ensures that the group labels do not appear in the index of the result. 
# .apply() will apply the given function to each group. 
# lambda is the other way of defining a function. E.g .shift(1) will take the Close column value from the previous row and assign it to the new column name Close_lag_1.

df_lags = df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Close_lag_2 = x['Adj Close'].shift(1))
)

dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(returns = (x['Close'] / x['Close_lag_1']) - 1,
                       hi_lo_range = x['High'] - x['Low'])
)

C:\Users\15199\AppData\Local\Temp\ipykernel_18320\3633471738.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_lags = df.groupby('Ticker', group_keys=False).apply(
C:\Users\15199\AppData\Local\Temp\ipykernel_18320\3633471738.py:13: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [10]:
# Converting Dask data frame to a pandas data frame. Further calculating moving average using a window of 10 days. First nine values will be NaN as there are not 10 values for the mean.
dd_feat['rolling_mean'] = dd_feat.compute()['returns'].rolling(10).mean()
dd_feat.compute()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_2,returns,hi_lo_range,rolling_mean
28730,2001-10-17 00:00:00+00:00,TRMB,2.841667,2.841667,3.000000,2.826667,2.951667,830400.0,2001,2.875000,2.875000,-0.011594,0.173333,0.018642
174883,2010-11-11 00:00:00+00:00,LRCX,1.923232,4.537000,4.569000,4.450000,4.493000,27988000.0,2010,4.620000,1.958417,-0.017965,0.119000,-0.000104
67171,2004-03-09 00:00:00+00:00,LRCX,0.991077,2.338000,2.388000,2.287000,2.380000,49335000.0,2004,2.383000,1.010153,-0.018884,0.101000,-0.004349
333290,2020-09-14 00:00:00+00:00,NTAP,40.408920,45.380001,45.669998,44.299999,44.389999,2788200.0,2020,44.090000,39.260223,0.029258,1.369999,-0.000371
159687,2009-12-03 00:00:00+00:00,AMD,8.030000,8.030000,8.230000,7.360000,7.370000,69019800.0,2009,7.300000,7.300000,0.100000,0.869999,0.009979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282276,2017-07-14 00:00:00+00:00,MCHP,35.641651,40.419998,40.509998,39.865002,39.939999,2835000.0,2017,39.744999,35.046452,0.016983,0.644997,0.134051
370020,2022-12-22 00:00:00+00:00,MCHP,67.036026,69.910004,71.370003,68.449997,71.370003,7644500.0,2022,72.959999,69.960632,-0.041804,2.920006,-0.009149
318740,2019-10-18 00:00:00+00:00,FICO,301.859985,301.859985,312.170013,297.420013,311.980011,297300.0,2019,311.980011,311.980011,-0.032438,14.750000,-0.002492
63316,2003-12-10 00:00:00+00:00,FICO,31.306667,32.266666,33.299999,32.099998,33.233334,1374750.0,2003,33.233334,32.244572,-0.029087,1.200001,-0.008140


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

+ It's not necessary to convert to pandas to calculate the moving average return. We can also work with the csv files. Also, repartition() can be used with map_overlap() without converting Dask DataFrame to pandas for calculating the rolling average.  
Dask DataFrame is lazy, meaning it doesn't compute results immediately. It does not show actual data; it just provides an overview. So, the other method is to use .compute() to show actual results to calculate the rolling average, and the compute method converts the dask DataFrame to pandas DataFrame. For data that fits into RAM, pandas can often be faster and easier to use than Dask DataFrame.  

+ It depends on the size of the dataset. If it's small, pandas is the better than Dask. However, if it's large dataset, Dask is the best choice. Our dataset is not very large, so converting to pandas is good choice.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.